In [1]:
import pandas as pd

df = pd.read_csv('Processed_Headings/combined.csv')
# df['sec-title'] = df.apply(lambda row: row['xmlPath'].rsplit(' > ', 1)[-1].strip(), axis=1)
# df.to_csv('Processed_Headings/combined.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,PMCID,PMID,xmlPath,sec-type,sec-title
0,0,0,PMC7000045,31567706,article > body > sec > Introduction,intro,Introduction
1,1,1,PMC7000045,31567706,article > body > sec > Patients and Methods,patients|methods,Patients and Methods
2,2,2,PMC7000045,31567706,article > body > sec > Results,results,Results
3,3,3,PMC7000045,31567706,article > body > sec > Discussion,discussion,Discussion
4,4,4,PMC7000113,31888964,article > body > sec > EXPERIMENTAL PROCEDURES,methods,EXPERIMENTAL PROCEDURES


In [26]:
len(df)

37642687

In [11]:

from sentence_transformers import SentenceTransformer, InputExample, losses
fine_tuned_model = SentenceTransformer("models/fine_tuned_sentence_bert_model_ContrastiveLoss")

# 測試新模型
embeddings = fine_tuned_model.encode(df['sec-title'].tolist()[7528537*4:])

In [12]:
import torch
import torch.nn as nn

# 重新定義 Classifier 類別
class Classifier(nn.Module):
    def __init__(self, embedding_dim, num_classes):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(embedding_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  

torch.serialization.add_safe_globals([Classifier])


In [13]:
import torch
import pickle
model = torch.load("classifier_model.pth", weights_only=False)

model.eval()  

def predict(model, new_embeddings):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 自動選擇 GPU 或 CPU
    model.to(device)  # 將模型移到對應設備
    new_embeddings = torch.tensor(new_embeddings, dtype=torch.float32).to(device)  # 轉換為 tensor 並移到設備

    with torch.no_grad():  # 禁用梯度計算，加速推理
        outputs = model(new_embeddings)  # 前向傳播
        predictions = torch.argmax(outputs, dim=1)  # 取得最大值索引（即預測標籤）

    return predictions.cpu().numpy() 

# 4. **測試新資料**
predictions_supervised = predict(model, new_embeddings=embeddings)


In [14]:
import numpy as np


print("Predicted labels:", predictions_supervised)


Predicted labels: [1 0 1 ... 1 0 0]


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np

averages = np.load('averages.npy')

similarity_matrix = cosine_similarity(embeddings, averages)
predicted_averages = np.argmax(similarity_matrix, axis=1) 
print("Predicted labels:", predicted_averages)



Predicted labels: [1 0 1 ... 1 0 0]


In [16]:
def get_acc(pred, y_test):
    return np.sum(y_test == pred) / len(y_test) * 100

get_acc(predicted_averages, predictions_supervised)

99.83990253620257

In [17]:
with open('Processed_Headings/predictions_supervised7528537-5.pkl', 'wb') as f:
    pickle.dump(predictions_supervised, f)
with open('Processed_Headings/predicted_averages7528537-5.pkl', 'wb') as f:
    pickle.dump(predicted_averages, f)

In [23]:
with open('Processed_Headings/predictions_supervised7528537.pkl', "rb") as f:
    data = pickle.load(f).tolist()
data

[1,
 0,
 2,
 3,
 0,
 0,
 0,
 2,
 2,
 2,
 3,
 1,
 1,
 0,
 0,
 0,
 1,
 3,
 3,
 0,
 0,
 0,
 2,
 2,
 2,
 3,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 3,
 3,
 1,
 1,
 2,
 3,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 2,
 0,
 3,
 1,
 1,
 2,
 3,
 0,
 1,
 3,
 1,
 2,
 3,
 3,
 3,
 2,
 3,
 1,
 1,
 0,
 2,
 3,
 1,
 1,
 0,
 2,
 3,
 1,
 1,
 2,
 1,
 1,
 0,
 2,
 1,
 3,
 1,
 1,
 1,
 2,
 3,
 1,
 1,
 1,
 1,
 2,
 3,
 0,
 1,
 1,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 0,
 1,
 1,
 0,
 3,
 1,
 2,
 2,
 3,
 1,
 0,
 0,
 1,
 1,
 3,
 1,
 1,
 0,
 3,
 2,
 2,
 2,
 3,
 1,
 2,
 3,
 1,
 0,
 2,
 3,
 3,
 3,
 1,
 1,
 2,
 2,
 3,
 1,
 2,
 2,
 2,
 0,
 3,
 2,
 3,
 1,
 2,
 4,
 4,
 1,
 1,
 1,
 0,
 1,
 2,
 0,
 0,
 3,
 0,
 3,
 1,
 0,
 2,
 3,
 3,
 1,
 0,
 2,
 3,
 3,
 1,
 0,
 2,
 3,
 3,
 1,
 0,
 2,
 3,
 3,
 1,
 0,
 2,
 3,
 3,
 1,
 0,
 2,
 3,
 3,
 1,
 0,
 2,
 3,
 3,
 1,
 0,
 2,
 3,
 3,
 1,
 0,
 2,
 3,
 3,
 3,
 1,
 1,
 3,
 1,
 3,
 0,
 2,
 2,
 3,
 1,
 1,
 1,
 1,
 2,
 3,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 3,
 0,
 0,
 1,
 1,
 1,
 2,
 3,
 0,
 1,
 1,
 1,
 1,


In [27]:
data = []

with open('Processed_Headings/predictions_supervised7528537.pkl', "rb") as f:
    data = pickle.load(f).tolist()

for i in range(2, 6):
    with open(f'Processed_Headings/predictions_supervised7528537-{i}.pkl', "rb") as f:
        data = data + pickle.load(f).tolist()

# 顯示讀取的資料
with open('Processed_Headings/predictions_supervised.pkl', 'wb') as f:
    pickle.dump(data, f)

In [28]:
data = []

with open('Processed_Headings/predicted_averages7528537.pkl', "rb") as f:
    data = pickle.load(f).tolist()

for i in range(2, 6):
    with open(f'Processed_Headings/predicted_averages7528537-{i}.pkl', "rb") as f:
        data = data + pickle.load(f).tolist()

# 顯示讀取的資料
with open('Processed_Headings/predicted_averages.pkl', 'wb') as f:
    pickle.dump(data, f)

In [32]:
with open('Processed_Headings/predicted_averages.pkl', "rb") as f:
    data = pickle.load(f)
len(data)

37642687

In [2]:
import pickle
averaged = []
with open('Processed_Headings/predicted_averages.pkl', "rb") as f:
    averaged = pickle.load(f)
len(averaged)

supervised = []
with open('Processed_Headings/predictions_supervised.pkl', "rb") as f:
    supervised = pickle.load(f)
len(supervised)

37642687

In [3]:
mapping = {
    0:'methods',
    1:'background',
    2:'results',
    3:'conclusions',
    4:'objective',
}

In [5]:
import pandas as pd

df = pd.read_csv('Processed_Headings/combined.csv')
# df['sec-title'] = df.apply(lambda row: row['xmlPath'].rsplit(' > ', 1)[-1].strip(), axis=1)
# df.to_csv('Processed_Headings/combined.csv')
df['average_prediction'] = [mapping[x] for x in averaged]
df['supervised_prediction'] = [mapping[x] for x in supervised]
df.to_csv('Processed_Headings/combined.csv')

In [6]:
df.head(40)

,Unnamed: 0.1,Unnamed: 0,PMCID,PMID,xmlPath,sec-type,sec-title,average_prediction,supervised_prediction
0,0,0,PMC7000045,31567706,article > body > sec > Introduction,intro,Introduction,background,background
1,1,1,PMC7000045,31567706,article > body > sec > Patients and Methods,patients|methods,Patients and Methods,methods,methods
2,2,2,PMC7000045,31567706,article > body > sec > Results,results,Results,results,results
3,3,3,PMC7000045,31567706,article > body > sec > Discussion,discussion,Discussion,conclusions,conclusions
4,4,4,PMC7000113,31888964,article > body > sec > EXPERIMENTAL PROCEDURES,methods,EXPERIMENTAL PROCEDURES,methods,methods
5,5,5,PMC7000113,31888964,article > body > sec > EXPERIMENTAL PROCEDURES,NaN,EXPERIMENTAL PROCEDURES,methods,methods
6,6,6,PMC7000113,31888964,article > body > sec > EXPERIMENTAL PROCEDURES,NaN,EXPERIMENTAL PROCEDURES,methods,methods
7,7,7,PMC7000113,31888964,article > body > sec > RESULTS,results,RESULTS,results,results
8,8,8,PMC7000113,31888964,article > body > sec > RESULTS,NaN,RESULTS,results,results
9,9,9,PMC7000113,31888964,article > body > sec > RESULTS,NaN,RESULTS,results,results
